###Imports

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_euse = pd.read_csv("/content/drive/MyDrive/capstone_data/use_all_btu.csv")

In [ ]:
df_msn_codes = pd.read_csv("/content/drive/MyDrive/capstone_data/Codes_and_Descriptions.csv")

In [ ]:
print(df_euse.describe())

               1960          1961          1962          1963          1964  \
count  1.144700e+04  1.144700e+04  1.144700e+04  1.144700e+04  1.144700e+04   
mean   9.962268e+04  1.011178e+05  1.056051e+05  1.097824e+05  1.142243e+05   
std    9.951270e+05  1.009306e+06  1.053619e+06  1.095018e+06  1.140634e+06   
min   -1.360010e+05 -1.402480e+05 -1.441310e+05 -1.475590e+05 -1.547940e+05   
25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
50%    1.465000e+03  1.427000e+03  1.514000e+03  1.545000e+03  1.670000e+03   
75%    2.246700e+04  2.273100e+04  2.374700e+04  2.474900e+04  2.558250e+04   
max    4.393528e+07  4.455770e+07  4.654313e+07  4.840968e+07  5.049417e+07   

               1965          1966          1967          1968          1969  \
count  1.144700e+04  1.144700e+04  1.144700e+04  1.144700e+04  1.144700e+04   
mean   1.189127e+05  1.254902e+05  1.295349e+05  1.376566e+05  1.444052e+05   
std    1.186977e+06  1.252849e+06  1.292601e+06  1.

In [ ]:
print(df_msn_codes.head())
print(df_msn_codes.info())
print(df_msn_codes.describe())

     MSN                                        Description  \
0  ABICB  Aviation gasoline blending components consumed...   
1  ABICP  Aviation gasoline blending components consumed...   
2  ARICB  Asphalt and road oil consumed by the industria...   
3  ARICD  Asphalt and road oil price in the industrial s...   
4  ARICP  Asphalt and road oil consumed by the industria...   

                      Unit  
0              Billion Btu  
1         Thousand barrels  
2              Billion Btu  
3  Dollars per million Btu  
4         Thousand barrels  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MSN          920 non-null    object
 1   Description  920 non-null    object
 2   Unit         920 non-null    object
dtypes: object(3)
memory usage: 21.7+ KB
None
          MSN                                        Description         Unit
count     920 

In [ ]:
# Merge
df_usage = pd.merge(df_euse, df_msn_codes, on="MSN", how="left")

In [ ]:
df_usage.head()

,Data_Status,State,MSN,1960,1961,1962,1963,1964,1965,1966,...,2016,2017,2018,2019,2020,2021,2022,2023,Description,Unit
0,2023F,AK,ABICB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.0,-2.0,-14.0,-10.0,-7.0,-8.0,-6.0,-6.0,Aviation gasoline blending components consumed...,Billion Btu
1,2023F,AK,ARICB,312.0,555.0,489.0,589.0,791.0,878.0,1646.0,...,12365.0,13561.0,4804.0,11973.0,12391.0,13425.0,12981.0,13173.0,Asphalt and road oil consumed by the industria...,Billion Btu
2,2023F,AK,ARTCB,312.0,555.0,489.0,589.0,791.0,878.0,1646.0,...,12365.0,13561.0,4804.0,11973.0,12391.0,13425.0,12981.0,13173.0,Asphalt and road oil total consumption,Billion Btu
3,2023F,AK,ARTXB,312.0,555.0,489.0,589.0,791.0,878.0,1646.0,...,12365.0,13561.0,4804.0,11973.0,12391.0,13425.0,12981.0,13173.0,Asphalt and road oil total end-use consumption,Billion Btu
4,2023F,AK,AVACB,5209.0,6441.0,5202.0,3273.0,2319.0,1478.0,1086.0,...,1240.0,1239.0,1264.0,1245.0,915.0,1037.0,1073.0,1019.0,Aviation gasoline consumed by the transportati...,Billion Btu


In [4]:
import pandas as pd

# === Load Data ===
energy_df = df_euse

# === Melt the wide format to long format (years as rows) ===
value_vars = [col for col in energy_df.columns if col.isdigit()]
melted = pd.melt(
    energy_df,
    id_vars=["MSN", "State"],
    value_vars=value_vars,
    var_name="Year",
    value_name="Data"
)

# === Convert data types ===
melted["Year"] = melted["Year"].astype(int)
melted["Data"] = pd.to_numeric(melted["Data"], errors="coerce")

# === Define prefixes ===
fossil_prefixes = {"NG", "CL", "CO", "PA", "PC", "DF", "JF", "FF"}
renewable_prefixes = {"HY", "WD", "WS", "SO", "GE", "WY"}

# === Filter for relevant MSN codes and classify energy types ===
melted = melted[melted["MSN"].str[:2].isin(fossil_prefixes.union(renewable_prefixes))]
melted["energy_type"] = melted["MSN"].str[:2].map(
    lambda x: "Fossil" if x in fossil_prefixes else "Renewable"
)

# === Aggregate total energy by State-Year-Type ===
agg_df = (
    melted.groupby(["State", "Year", "energy_type"])["Data"]
    .sum()
    .reset_index()
    .pivot(index=["State", "Year"], columns="energy_type", values="Data")
    .fillna(0)
    .reset_index()
)

# === Compute fossil fuel share ===
agg_df["Total"] = agg_df["Fossil"] + agg_df["Renewable"]
agg_df["fossil_share"] = agg_df["Fossil"] / agg_df["Total"]

# === Compare fossil share in 2000 vs 2019 ===
df_2000 = agg_df[agg_df["Year"] == 2000][["State", "fossil_share"]].rename(columns={"fossil_share": "fossil_share_2000"})
df_2019 = agg_df[agg_df["Year"] == 2019][["State", "fossil_share"]].rename(columns={"fossil_share": "fossil_share_2019"})
merged = pd.merge(df_2000, df_2019, on="State")

# === Compute change in fossil share and label success ===
merged["delta_fossil_share"] = merged["fossil_share_2000"] - merged["fossil_share_2019"]
median_delta = merged["delta_fossil_share"].median()
merged["Y_OUTCOME"] = (merged["delta_fossil_share"] > median_delta).astype(int)

# === Preview results ===
print(merged.head())

# === Optionally save to file ===
merged.to_csv("preprocessed_fossil_reduction_2000_2019.csv", index=False)

energy_type State  fossil_share_2000  fossil_share_2019  delta_fossil_share  \
0              AK           0.996896           0.990457            0.006439   
1              AL           0.940522           0.938843            0.001679   
2              AR           0.951562           0.953777           -0.002215   
3              AZ           0.980454           0.975013            0.005441   
4              CA           0.973189           0.958252            0.014937   

energy_type  Y_OUTCOME  
0                    0  
1                    0  
2                    0  
3                    0  
4                    1  


In [12]:
merged

energy_type,State,fossil_share_2000,fossil_share_2019,delta_fossil_share,Y_OUTCOME
0,AK,0.996896,0.990457,0.006439,0
1,AL,0.940522,0.938843,0.001679,0
2,AR,0.951562,0.953777,-0.002215,0
3,AZ,0.980454,0.975013,0.005441,0
4,CA,0.973189,0.958252,0.014937,1
5,CO,0.992738,0.975435,0.017303,1
6,CT,0.967748,0.977812,-0.010064,0
7,DC,0.991135,0.984993,0.006142,0
8,DE,0.995219,0.993608,0.001611,0
9,FL,0.968745,0.970516,-0.001771,0


In [5]:
agg_df

energy_type,State,Year,Fossil,Renewable,Total,fossil_share
0,AK,1960,299632.2,9339.0,308971.2,0.969774
1,AK,1961,355776.0,10321.0,366097.0,0.971808
2,AK,1962,391960.0,10564.0,402524.0,0.973756
3,AK,1963,399362.2,10985.0,410347.2,0.973230
4,AK,1964,420770.9,11656.0,432426.9,0.973045
...,...,...,...,...,...,...
3323,WY,2019,2907712.5,49914.0,2957626.5,0.983124
3324,WY,2020,2704450.1,54141.8,2758591.9,0.980373
3325,WY,2021,2688048.9,72887.9,2760936.8,0.973600
3326,WY,2022,2767250.2,82465.1,2849715.3,0.971062


In [7]:
agg_df = agg_df[agg_df['Year'] >= 2000]
agg_df = agg_df[agg_df['Year'] <= 2019]

In [8]:
agg_df

energy_type,State,Year,Fossil,Renewable,Total,fossil_share
40,AK,2000,3497341.0,10891.0,3508232.0,0.996896
41,AK,2001,3438696.1,15393.0,3454089.1,0.995544
42,AK,2002,3425115.5,16466.0,3441581.5,0.995216
43,AK,2003,3445911.9,17590.0,3463501.9,0.994921
44,AK,2004,3744150.3,17134.0,3761284.3,0.995445
...,...,...,...,...,...,...
3319,WY,2015,2959612.6,44722.5,3004335.1,0.985114
3320,WY,2016,2865241.5,48700.1,2913941.6,0.983287
3321,WY,2017,2983055.2,50626.2,3033681.4,0.983312
3322,WY,2018,3087247.3,47750.1,3134997.4,0.984769


In [9]:
agg_df = agg_df[agg_df['State'] != "US"]
agg_df = agg_df[agg_df['State'] != "DC"]

In [10]:
agg_df

energy_type,State,Year,Fossil,Renewable,Total,fossil_share
40,AK,2000,3497341.0,10891.0,3508232.0,0.996896
41,AK,2001,3438696.1,15393.0,3454089.1,0.995544
42,AK,2002,3425115.5,16466.0,3441581.5,0.995216
43,AK,2003,3445911.9,17590.0,3463501.9,0.994921
44,AK,2004,3744150.3,17134.0,3761284.3,0.995445
...,...,...,...,...,...,...
3319,WY,2015,2959612.6,44722.5,3004335.1,0.985114
3320,WY,2016,2865241.5,48700.1,2913941.6,0.983287
3321,WY,2017,2983055.2,50626.2,3033681.4,0.983312
3322,WY,2018,3087247.3,47750.1,3134997.4,0.984769


In [11]:
# Export to CSV (save to Google Drive)
agg_df.to_csv('/content/drive/My Drive/capstone_data_cleaned/df_eusage.csv', index=False)